## Análise das partidas de um jogador na plataforma Chess.com


Neste projeto buscamos fazer uma análise exploratória dos dados de todas as partidas do Grande Mestre Evandro Barbosa na plataforma Chess.com com o objetivo de responder às seguintes questões: 

1. O jogador joga melhor de Pretas, de Brancas, ou isso depende de algum outro fator?
2. Quais as relações entre o rating (pontuação) de um oponente e o resultado da partida? Por exemplo, será que necessariamente o jogador tende a empatar com jogadores com rating mais próximo ao seu?

O Pojeto está dividido nas seguintes partes: 
- Extração e transformação dos dados;
- Análise exploratória com o objetivo de identificar padrões e relações pertinentes;
- Respostas às perguntas supracitadas

#### Importação:

Iniciamos importando os módulos, bibliotecas e funções usados:

In [19]:
import concurrent.futures
from datetime import datetime

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import transform.chess_transform as ct
from extract.chess_request import request_games
from extract.chess_request import get_user

### <u>Extração e Transformação de Dados</u>

Os dados serão extraídos da API do Chess.com. A API tem as partidas de cada mês em endpoints separados. Para pegar todas as partidas de Julho de 2018 para o dia de hoje, devemos fazer várias requisições à API, e isso pode custar tempo. Para acelerar este processo, criamos a função abaixo para fazer requisições de maneira concorrente:

In [2]:
def worker(year_,user_,email_):
	df_list = []
	current_month = datetime.now().month
	global current_year

	for month in range(1, 13):
		if (year_ == 2018) and (month < 7):
			return None
		df = request_games(year_, month, user_, email_)
		df_list += [df]
		if (year_ == current_year) and (month == current_month):
			break
	return pd.concat(df_list)

Abaixo extraímos os dados:

In [3]:
user = "Evandro_Barbosa" # Grande mestre
current_year = datetime.now().year

# preenchendo a variável email com o conteúdo de um arquivo oculto
with open("extract/.email", 'r') as email_file:
	email = email_file.read().strip()

# fazendo requisições em filas concorrentes (uma para cada ano)
with concurrent.futures.ThreadPoolExecutor() as executor:
	futures = []
	for year in range(2018, current_year+1):
			futures += [executor.submit(worker, year, user, email)]
	df_list = [f.result() for f in futures]

df = pd.concat(df_list)    

requesting game data from 12/2021...  

Abaixo aplicamos algumas transformações, salvamos o DataFrame em um arquivo CSV para uso futuro e, finalmente, exibimos nosso conjunto de dados:

In [4]:
df = ct.adequate(df)
df.sort_index(inplace=True)
df   = ct.personalize(df, user)

df.to_csv(f"{user}.csv")

df

,time_control,time_class,rating,played_as,result,against,opponent_rating,opponent,opponent_result,opponent_uuid,rated,url,draw
end_time,,,,,,,,,,,,,
2019-01-02 08:03:43,180,blitz,2572,black,checkmated,white,2580,Timerkhanov,win,ffcb87e4-34af-11e7-8000-000000000000,True,https://www.chess.com/game/live/3342861344,False
2019-01-02 08:11:18,180,blitz,2567,black,resigned,white,2700,Aleksey_Sorokin,win,42236744-6ac8-11e7-8006-000000000000,True,https://www.chess.com/game/live/3342874139,False
2019-01-02 08:52:23,180,blitz,2576,black,win,white,2580,Chuson,resigned,be23857e-8fbb-11e6-8012-000000000000,True,https://www.chess.com/game/live/3342946274,False
2019-01-02 08:55:21,180,blitz,2570,white,resigned,black,2681,jerom555888,win,a78f54ae-b630-11e7-8062-000000000000,True,https://www.chess.com/game/live/3342949166,False
2019-01-02 08:58:56,180,blitz,2580,black,win,white,2671,jerom555888,resigned,a78f54ae-b630-11e7-8062-000000000000,True,https://www.chess.com/game/live/3342953916,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-02-25 13:11:25,180,blitz,2705,black,win,white,2686,AnonForever,resigned,71bf3ede-c146-11ee-a523-9d932363b9f1,True,https://www.chess.com/game/live/102616266465,False
2024-02-25 14:51:19,180,blitz,2712,black,win,white,2669,H0UGH,resigned,76eda048-d194-11eb-9477-cd60389704ad,True,https://www.chess.com/game/live/102622224113,False
2024-02-25 15:01:29,180,blitz,2720,black,win,white,2685,JesseKraai,resigned,b5a989f8-3446-11e3-8006-000000000000,True,https://www.chess.com/game/live/102622824033,False


In [12]:
df.columns

Index(['time_control', 'time_class', 'rating', 'played_as', 'result',
       'against', 'opponent_rating', 'opponent', 'opponent_result',
       'opponent_uuid', 'rated', 'url', 'draw'],
      dtype='object')

Acima, nota-se que não há partidas anteriores a 2019, embora não esteja ocorrendo erro nas requisições. Vamos trabalhar com os dados que temos, que são suficientes para uma análise. 

Vamos começar entendendo as colunas de que dispomos: 
- `'time_control'`: quantidade de segundos disponíveis no relógio de cada jogador no início da partida. Pode ser um número apenas, como 180 (indicando partidas de 3 minutos), ou algo como "60+1", que indica que cada jogador tem 1 minuto no relógio e ganha mais 1 segundo a cada lance.
- `'time_class'`: Categoria do jogo, classificado a partir do tempo de duração da partida. 
- `'rating'` e `'opponent_rating`': Pontuação do jogador, naquela categoria, ao final da partida. Jogadores tem ratings diferentes para categorias (valores de 'time_class') distintas.
- `'played_as'` e `'against'`: Cores das peças do Jogador e de seu oponente, respectivamente. 
- `'opponent'` e `'opponent_uuid'`: Nome de usuário e uuid do oponente, respectivamente.
- `'result'` e `'opponent_result'`: resultado da partida para cada jogador. "Win" indica que o jogador ou o oponente (a depender da coluna em que se encontra) venceu a partida. Em caso de derrota ou empate, a coluna consta com o valor que representa do motivo do empate.
- `'rated'`: indica se a partida estava valendo pontos.
- `url`: url da partida, onde pode se observar todos os seus lances.
- `'draw'`: indica se a partida terminou ou não em empate.

Abaixo, podemos ver o controle de tempo das partidas que o jogador disputou em cada categoria.

In [13]:
for category in df['time_class'].unique():
    print(category+":\t",df[df['time_class'] == category]['time_control'].unique())

blitz:	 ['180' '600' '300' '180+2' '300+5' '60+3' '300+2' '180+1']
bullet:	 ['60' '30' '120' '10']
rapid:	 ['60+15' '1260' '1020' '1260+9' '900' '900+10' '600+10' '1500' '600']
